<a href="https://colab.research.google.com/github/dalatorrem/Diplomado_Metodos_UCentral_2023/blob/main/28_practica_analisis_lineal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Modelo de predicción del puntaje en la prueba saber 11

Contexto Analítico: construir un modelo lineal que teniendo como variables de entrada variables sociodemográficas nos permita dar una predicción del puntaje global.


In [1]:
import pandas                   as pd
import numpy                    as np
import matplotlib.pyplot        as plt
import seaborn                  as sns
from   scipy                    import stats
import statsmodels.formula.api  as smf
import scipy
import warnings
from sklearn.model_selection    import train_test_split
from sklearn.linear_model       import LinearRegression
from sklearn.metrics            import mean_squared_error, mean_absolute_error
from sklearn.metrics            import mean_absolute_percentage_error
import plotly.express as px

In [2]:
path_data = 'https://raw.githubusercontent.com/dalatorrem/Diplomado_Metodos_UCentral_2023/main/data/Resultados__nicos_Saber_11_sample.csv'
data_saber = pd.read_csv(path_data)
data_saber

,Unnamed: 0,PERIODO,ESTU_TIPODOCUMENTO,ESTU_CONSECUTIVO,COLE_AREA_UBICACION,COLE_BILINGUE,COLE_CALENDARIO,COLE_CARACTER,COLE_COD_DANE_ESTABLECIMIENTO,COLE_COD_DANE_SEDE,...,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET,FAMI_TIENELAVADORA,DESEMP_INGLES,PUNT_INGLES,PUNT_MATEMATICAS,PUNT_SOCIALES_CIUDADANAS,PUNT_C_NATURALES,PUNT_LECTURA_CRITICA,PUNT_GLOBAL
0,0,20131,CR,SB11201310000414,URBANO,N,B,ACADÉMICO,3.118480e+11,3.118480e+11,...,Si,Si,Si,B+,94.0,88.0,NaN,NaN,NaN,NaN
1,1,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,1.410160e+11,2.410160e+11,...,Si,Si,Si,B1,71.0,66.0,70.0,65.0,69.0,339.0
2,2,20194,TI,SB11201940464873,RURAL,N,A,TÉCNICO/ACADÉMICO,1.410160e+11,2.410160e+11,...,Si,Si,Si,B1,71.0,66.0,70.0,65.0,69.0,339.0
3,3,20122,TI,SB11201220204399,URBANO,N,A,TÉCNICO/ACADÉMICO,1.631300e+11,1.631300e+11,...,Si,Si,No,A1,48.0,45.0,NaN,NaN,NaN,NaN
4,4,20132,TI,SB11201320464198,URBANO,N,A,TÉCNICO,1.190010e+11,1.190010e+11,...,Si,Si,Si,A-,43.0,52.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,20162,TI,SB11201620287491,RURAL,NaN,A,ACADÉMICO,2.687200e+11,2.687200e+11,...,No,No,No,A1,55.0,50.0,57.0,48.0,53.0,262.0
9996,9996,20132,TI,SB11201320276289,URBANO,N,A,TÉCNICO/ACADÉMICO,2.470010e+11,2.470010e+11,...,Si,Si,No,A-,39.0,57.0,NaN,NaN,NaN,NaN
9997,9997,20194,TI,SB11201940202131,RURAL,N,A,ACADÉMICO,2.735040e+11,2.735040e+11,...,No,No,No,A-,36.0,36.0,52.0,51.0,55.0,238.0
9998,9998,20194,TI,SB11201940202131,RURAL,N,A,ACADÉMICO,2.735040e+11,2.735040e+11,...,No,No,No,A-,36.0,36.0,52.0,51.0,55.0,238.0


In [3]:
data_saber.shape

(10000, 52)

In [4]:
TARGET = 'PUNT_GLOBAL'

# OJO con las variables a usar

In [6]:
data_saber.columns

Index(['Unnamed: 0', 'PERIODO', 'ESTU_TIPODOCUMENTO', 'ESTU_CONSECUTIVO',
       'COLE_AREA_UBICACION', 'COLE_BILINGUE', 'COLE_CALENDARIO',
       'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE',
       'COLE_COD_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION', 'COLE_GENERO',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA',
       'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_ESTUDIANTE', 'ESTU_FECHANACIMIENTO',
       'ESTU_GENERO', 'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE',
       'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD',
       'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA

In [7]:
data_modelo_1 = data_saber[['PUNT_INGLES', 'PUNT_MATEMATICAS',
       'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_LECTURA_CRITICA',
       'PUNT_GLOBAL']].copy()
data_modelo_1 = data_modelo_1.dropna()
data_modelo_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5619 entries, 1 to 9999
Data columns (total 6 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   PUNT_INGLES               5619 non-null   float64
 1   PUNT_MATEMATICAS          5619 non-null   float64
 2   PUNT_SOCIALES_CIUDADANAS  5619 non-null   float64
 3   PUNT_C_NATURALES          5619 non-null   float64
 4   PUNT_LECTURA_CRITICA      5619 non-null   float64
 5   PUNT_GLOBAL               5619 non-null   float64
dtypes: float64(6)
memory usage: 307.3 KB


In [8]:
formula_1 = f'{TARGET} ~ PUNT_INGLES + PUNT_MATEMATICAS + PUNT_SOCIALES_CIUDADANAS + PUNT_C_NATURALES + PUNT_LECTURA_CRITICA'
model_no_usable = smf.ols(formula=formula_1,data=data_modelo_1).fit()
print(model_no_usable.summary())

                            OLS Regression Results                            
Dep. Variable:            PUNT_GLOBAL   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.676e+07
Date:                Wed, 30 Aug 2023   Prob (F-statistic):               0.00
Time:                        02:02:43   Log-Likelihood:                -2887.6
No. Observations:                5619   AIC:                             5787.
Df Residuals:                    5613   BIC:                             5827.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

**Este modelo no aporta por dos razones:**

1. La variable de salida está en función de las variables de entrada, especificamente se sabe que el puntaje global es una combinación de los demás puntajes.
2. Los puntajes por componente aparecen al tiempo del puntaje global

# OJo con los códigos

In [9]:
data_saber.columns

Index(['Unnamed: 0', 'PERIODO', 'ESTU_TIPODOCUMENTO', 'ESTU_CONSECUTIVO',
       'COLE_AREA_UBICACION', 'COLE_BILINGUE', 'COLE_CALENDARIO',
       'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE',
       'COLE_COD_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION', 'COLE_GENERO',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA',
       'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_ESTUDIANTE', 'ESTU_FECHANACIMIENTO',
       'ESTU_GENERO', 'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE',
       'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD',
       'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA

In [10]:
data_modelo_2 = data_saber[['ESTU_CONSECUTIVO', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE', 'COLE_COD_DEPTO_UBICACION',
       'PUNT_GLOBAL']].copy()
data_modelo_2 = data_modelo_2.dropna()
data_modelo_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5619 entries, 1 to 9999
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ESTU_CONSECUTIVO               5619 non-null   object 
 1   COLE_COD_DANE_ESTABLECIMIENTO  5619 non-null   float64
 2   COLE_COD_DANE_SEDE             5619 non-null   float64
 3   COLE_COD_DEPTO_UBICACION       5619 non-null   int64  
 4   PUNT_GLOBAL                    5619 non-null   float64
dtypes: float64(3), int64(1), object(1)
memory usage: 263.4+ KB


In [ ]:
formula_2 = f'{TARGET} ~ ESTU_CONSECUTIVO + COLE_COD_DANE_ESTABLECIMIENTO + COLE_COD_DANE_SEDE + COLE_COD_DEPTO_UBICACION'
model_no_usable = smf.ols(formula=formula_2,data=data_modelo_2).fit()
print(model_no_usable.summary())

**Las variables COLE_COD_DANE_ESTABLECIMIENTO + COLE_COD_DANE_SEDE + COLE_COD_DEPTO_UBICACION son variables categóricos y deben ser tratadas de esa forma**

**La columna ESTU_CONSECUTIVO no es una variable, es un código de identificación, por lo tanto seguramente es único**

# Selección de variables de acuerdo al conocimiento del equipo que modela:

In [5]:
list_vars_model = ['Unnamed: 0', 'PERIODO', 'ESTU_TIPODOCUMENTO', 'ESTU_CONSECUTIVO',
       'COLE_AREA_UBICACION', 'COLE_BILINGUE', 'COLE_CALENDARIO',
       'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE',
       'COLE_COD_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION', 'COLE_GENERO',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA',
       'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_ESTUDIANTE', 'ESTU_FECHANACIMIENTO',
       'ESTU_GENERO', 'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE',
       'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD',
       'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_PERSONASHOGAR', 'FAMI_TIENEAUTOMOVIL',
       'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA',
       'DESEMP_INGLES', 'PUNT_INGLES', 'PUNT_MATEMATICAS',
       'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_LECTURA_CRITICA',
       'PUNT_GLOBAL']

In [6]:
data_saber['COLE_NATURALEZA'].unique()

array(['NO OFICIAL', 'OFICIAL'], dtype=object)

Resultados 'DESEMP_INGLES', 'PUNT_INGLES', 'PUNT_MATEMATICAS',
       'PUNT_SOCIALES_CIUDADANAS', 'PUNT_C_NATURALES', 'PUNT_LECTURA_CRITICA',
       'PUNT_GLOBAL'

No es variable, es constante ESTU_ESTUDIANTE

Error al guardar 'Unnamed: 0'

PERIODO no se utilizará para este análisis

Identificador, no es variable ESTU_CONSECUTIVO

In [7]:
list_vars_model = [ 'ESTU_TIPODOCUMENTO',
       'COLE_AREA_UBICACION', 'COLE_BILINGUE', 'COLE_CALENDARIO',
       'COLE_CARACTER', 'COLE_COD_DANE_ESTABLECIMIENTO', 'COLE_COD_DANE_SEDE',
       'COLE_COD_DEPTO_UBICACION', 'COLE_COD_MCPIO_UBICACION',
       'COLE_CODIGO_ICFES', 'COLE_DEPTO_UBICACION', 'COLE_GENERO',
       'COLE_JORNADA', 'COLE_MCPIO_UBICACION', 'COLE_NATURALEZA',
       'COLE_NOMBRE_ESTABLECIMIENTO', 'COLE_NOMBRE_SEDE',
       'COLE_SEDE_PRINCIPAL', 'ESTU_COD_DEPTO_PRESENTACION',
       'ESTU_COD_MCPIO_PRESENTACION', 'ESTU_COD_RESIDE_DEPTO',
       'ESTU_COD_RESIDE_MCPIO', 'ESTU_DEPTO_PRESENTACION', 'ESTU_DEPTO_RESIDE',
       'ESTU_ESTADOINVESTIGACION', 'ESTU_FECHANACIMIENTO',
       'ESTU_GENERO', 'ESTU_MCPIO_PRESENTACION', 'ESTU_MCPIO_RESIDE',
       'ESTU_NACIONALIDAD', 'ESTU_PAIS_RESIDE', 'ESTU_PRIVADO_LIBERTAD',
       'FAMI_CUARTOSHOGAR', 'FAMI_EDUCACIONMADRE', 'FAMI_EDUCACIONPADRE',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_PERSONASHOGAR', 'FAMI_TIENEAUTOMOVIL',
       'FAMI_TIENECOMPUTADOR', 'FAMI_TIENEINTERNET', 'FAMI_TIENELAVADORA',
       ]

In [8]:
list_cols = list_vars_model + [TARGET]
data_modelo_3 = data_saber[list_cols].dropna().copy()
data_modelo_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4611 entries, 1 to 9999
Data columns (total 42 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ESTU_TIPODOCUMENTO             4611 non-null   object 
 1   COLE_AREA_UBICACION            4611 non-null   object 
 2   COLE_BILINGUE                  4611 non-null   object 
 3   COLE_CALENDARIO                4611 non-null   object 
 4   COLE_CARACTER                  4611 non-null   object 
 5   COLE_COD_DANE_ESTABLECIMIENTO  4611 non-null   float64
 6   COLE_COD_DANE_SEDE             4611 non-null   float64
 7   COLE_COD_DEPTO_UBICACION       4611 non-null   int64  
 8   COLE_COD_MCPIO_UBICACION       4611 non-null   int64  
 9   COLE_CODIGO_ICFES              4611 non-null   float64
 10  COLE_DEPTO_UBICACION           4611 non-null   object 
 11  COLE_GENERO                    4611 non-null   object 
 12  COLE_JORNADA                   4611 non-null   o

In [9]:
data_modelo_3['COLE_AREA_UBICACION'].unique()

array(['RURAL', 'URBANO'], dtype=object)

## No realizar el análisis de este modo porque variables con mayor número de categorias pueden dar un r2 más alto sin ser buenas predictoras

In [10]:
data_modelo_3[list_vars_model] = data_modelo_3[list_vars_model].astype('str')
data_modelo_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4611 entries, 1 to 9999
Data columns (total 42 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   ESTU_TIPODOCUMENTO             4611 non-null   object 
 1   COLE_AREA_UBICACION            4611 non-null   object 
 2   COLE_BILINGUE                  4611 non-null   object 
 3   COLE_CALENDARIO                4611 non-null   object 
 4   COLE_CARACTER                  4611 non-null   object 
 5   COLE_COD_DANE_ESTABLECIMIENTO  4611 non-null   object 
 6   COLE_COD_DANE_SEDE             4611 non-null   object 
 7   COLE_COD_DEPTO_UBICACION       4611 non-null   object 
 8   COLE_COD_MCPIO_UBICACION       4611 non-null   object 
 9   COLE_CODIGO_ICFES              4611 non-null   object 
 10  COLE_DEPTO_UBICACION           4611 non-null   object 
 11  COLE_GENERO                    4611 non-null   object 
 12  COLE_JORNADA                   4611 non-null   o

In [11]:
for col in list_vars_model:
  model_col = smf.ols(formula=f'{TARGET}~{col}',data=data_modelo_3).fit()
  print(f'Modelo con sólo la variable {col} tiene un R2 de : {round(model_col.rsquared, 3)}')

Modelo con sólo la variable ESTU_TIPODOCUMENTO tiene un R2 de : 0.057
Modelo con sólo la variable COLE_AREA_UBICACION tiene un R2 de : 0.01
Modelo con sólo la variable COLE_BILINGUE tiene un R2 de : 0.021
Modelo con sólo la variable COLE_CALENDARIO tiene un R2 de : 0.057
Modelo con sólo la variable COLE_CARACTER tiene un R2 de : 0.007
Modelo con sólo la variable COLE_COD_DANE_ESTABLECIMIENTO tiene un R2 de : 0.798
Modelo con sólo la variable COLE_COD_DANE_SEDE tiene un R2 de : 0.803
Modelo con sólo la variable COLE_COD_DEPTO_UBICACION tiene un R2 de : 0.073
Modelo con sólo la variable COLE_COD_MCPIO_UBICACION tiene un R2 de : 0.259
Modelo con sólo la variable COLE_CODIGO_ICFES tiene un R2 de : 0.865
Modelo con sólo la variable COLE_DEPTO_UBICACION tiene un R2 de : 0.074
Modelo con sólo la variable COLE_GENERO tiene un R2 de : 0.023
Modelo con sólo la variable COLE_JORNADA tiene un R2 de : 0.141
Modelo con sólo la variable COLE_MCPIO_UBICACION tiene un R2 de : 0.27
Modelo con sólo la va